In [ ]:

# Work 15: Modified Charlson Comorbidity Index (CCI) Calculation and ICD-10 Code Expansion [W15.CCI.2.mo_CCI_weights.ipynb]:

# "This notebook expands ICD-10 codes, assigns weights for CCI conditions, and saves the data to a CSV file." 

########################################################################################################
#  Sequence list
########################################################################################################

# 1: Helper function to expand ranges and exclude specific codes
# 1.1: Check if there is a range to expand
# 1.2: If it's a single code, return it unless excluded

# 2: Define corrected CCI ICD-10 codes and their weights

# 3: Convert the dictionary to a DataFrame for saving, expanding ranges
# 3.1: Define codes to exclude for 'Cancer' condition
# 3.2: Expand ICD-10 code ranges and exclude specific codes
# 3.3: Append expanded codes to the conditions list

# 4: Display the DataFrame in your environment (optional print)

# 5: Define the path to save the conditions

# 6: Save the DataFrame to a CSV file
# 6.1: Print confirmation message

########################################################################################################
########################################################################################################

import pandas as pd

# 1: Helper function to expand ranges and exclude specific codes
def expand_code_range(code_range, exclude_codes=None):
    """
    Expands a range of ICD-10 codes. If the code is not a range, returns it directly.
    Optionally excludes specific codes from the result.

    Parameters:
    code_range (str): A single ICD-10 code or a range of codes (e.g., 'I60-I69').
    exclude_codes (list): A list of codes to exclude from the expanded range.

    Returns:
    list: A list of ICD-10 codes after expansion and exclusion.
    """
    if exclude_codes is None:
        exclude_codes = []

    # 1.1: Check if there is a range to expand
    if '-' in code_range:
        start_code, end_code = code_range.split('-')
        start_prefix = ''.join(filter(str.isalpha, start_code))
        start_num = int(''.join(filter(str.isdigit, start_code)))
        end_num = int(''.join(filter(str.isdigit, end_code)))

        expanded_codes = []
        for num in range(start_num, end_num + 1):
            expanded_code = f"{start_prefix}{num}"
            # Handle cases where leading zeros might be needed
            if len(end_code) > len(expanded_code):
                expanded_code = f"{start_prefix}{str(num).zfill(len(end_code) - len(start_prefix))}"
            if expanded_code not in exclude_codes:
                expanded_codes.append(expanded_code)
        return expanded_codes

    # 1.2: If it's a single code, return it unless excluded
    return [code_range] if code_range not in exclude_codes else []

# 2: Define corrected CCI ICD-10 codes and their weights
cci_conditions = {
    'Acute myocardial infarction': {'weight': 1, 'codes': ['I21', 'I252']},  
    'Congestive heart failure': {'weight': 1, 'codes': ['I50']},
    'Peripheral vascular disease': {'weight': 1, 'codes': ['I702', 'I73']},
    'Cerebral vascular accident': {'weight': 1, 'codes': ['I60-I64', 'I69']},  # Exclude hemiplegia (handled separately)
    'Dementia': {'weight': 1, 'codes': ['F00', 'F01', 'F03', 'G30']},
    'Pulmonary disease': {'weight': 1, 'codes': ['J42-J47', 'J60-J67', 'J701', 'J703']},  # Chronic
    'Connective tissue disease': {'weight': 1, 'codes': ['M05', 'M06', 'M30-M36', 'M45']},
    'Peptic ulcer': {'weight': 1, 'codes': ['K25-K28']},
    'Liver disease': {'weight': 1, 'codes': ['B18', 'K704', 'K711', 'K713', 'K714', 'K715', 'K73', 'Z944']},
    'Severe liver disease': {'weight': 3, 'codes': ['K703', 'K717', 'K721', 'K729', 'K743-K746', 'I85', 'I864', 'I982']},
    'Diabetes without complication (mild diabetes mellitus)': {'weight': 1, 'codes': [
        'E100', 'E101', 'E106', 'E108', 'E109', 'E110', 'E111', 'E116', 'E118', 'E119',
        'E120', 'E121', 'E126', 'E128', 'E129', 'E130', 'E131', 'E136', 'E138', 'E139',
        'E140', 'E141', 'E146', 'E148', 'E149'
    ]},
    'Diabetes with complication (severe diabetes mellitus)': {'weight': 2, 'codes': [
        'E102', 'E103', 'E104', 'E105', 'E107', 'E112', 'E113', 'E114', 'E115', 'E117',
        'E122', 'E123', 'E124', 'E125', 'E127', 'E132', 'E133', 'E134', 'E135', 'E137',
        'E142', 'E143', 'E144', 'E145', 'E147'
    ]},
    'Hemiplegia': {'weight': 2, 'codes': ['G041', 'G114', 'G801', 'G81', 'G82', 'G830-G834', 'G839']},
    'Renal disease': {'weight': 2, 'codes': ['N18', 'Z940', 'Z491', 'Z492', 'Z992', 'T861']},  # Chronic
    'Cancer': {'weight': 2, 'codes': ['C00-C97']},  # Exclude C77-C80 later
    'Cancer with metastasis': {'weight': 6, 'codes': ['C77-C80']},
    'AIDS': {'weight': 6, 'codes': ['B20-B22', 'B24']}
}

# 3: Convert the dictionary to a DataFrame for saving, expanding ranges
# 3.1: Define codes to exclude for 'Cancer' condition
exclude_codes = ['C77', 'C78', 'C79', 'C80']

cci_conditions_list = []
for condition, details in cci_conditions.items():
    for code in details['codes']:
        # 3.2: Expand ICD-10 code ranges and exclude specific codes
        expanded_codes = expand_code_range(code, exclude_codes if condition == 'Cancer' else None)
        for expanded_code in expanded_codes:
            # 3.3: Append expanded codes to the conditions list
            cci_conditions_list.append({'Condition': condition, 'Weight': details['weight'], 'ICD-10 Code': expanded_code})

cci_conditions_df = pd.DataFrame(cci_conditions_list)

# 4: Display the DataFrame in your environment (optional print)
print(cci_conditions_df)

# 5: Define the path to save the conditions
cci_conditions_path = 'W15.mo_cci_conditions.csv'

# 6: Save the DataFrame to a CSV file
cci_conditions_df.to_csv(cci_conditions_path, index=False)

print(f"6: CCI conditions saved to '{cci_conditions_path}'.")
